#### Loading Necessary Packages

In [105]:
#%pip install lightgbm
#%pip install xgboost
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from scipy.stats import chi2_contingency
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from scipy.stats import f_oneway
from scipy.stats import pearsonr
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_auc_score, roc_curve, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
import lightgbm as lgb  
import xgboost as xgb   
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, cross_val_predict, StratifiedKFold

#### Loading Original Dataset

In [106]:
# Original Datafile is called Raw Data.
raw_data = pd.read_csv('SEO CRSP 2000 - 2009.csv')
raw_data

,Obs,CRSP_date,CUSIP6,NCUSIP,CRSP_TICKER,PERMNO,PERMCO,PRC,SHROUT,CUSIP,...,Secondary__Shs_Ofrd___sum_of_all,Sec_Shs__as___of_Shs__Ofrd___sum,Sec_Shs__as___of_Shs_Ofrd___in__,Secondary__Shs_Ofrd____in_this_0,Serial,Shareholder_Take_Up_All_Markets_,Shares_Ofrd_as___of_Shs_Out_Bef_,Total_Shares_Offered__mil_,Simul__taneous_Offer_Cusip,Stock_Price_at_Close_of_Offer__0
0,1,.,NaN,NaN,NaN,.,.,.,.,14846C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,.,NaN,NaN,NaN,.,.,.,.,18886P,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,.,NaN,NaN,NaN,.,.,.,.,29353W,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,.,NaN,NaN,NaN,.,.,.,.,42212J,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,.,NaN,NaN,NaN,.,.,.,.,46688X,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10438,10439,.,NaN,NaN,NaN,.,.,.,.,84663K,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.751,NaN,NaN
10439,10440,.,NaN,NaN,NaN,.,.,.,.,88588P,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.149,NaN,NaN
10440,10441,.,NaN,NaN,NaN,.,.,.,.,941845,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.000,NaN,NaN
10441,10442,.,NaN,NaN,NaN,.,.,.,.,973146,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.833,NaN,NaN


#### Initial Descriptive Analysis

In [107]:
# Assuming the dataset is loaded as shown in your original code
# Get unique marketplaces
unique_marketplaces = raw_data['Marketplace'].unique()

# Create a DataFrame for better table display
marketplace_df = pd.DataFrame({
    'Marketplace': unique_marketplaces
})

# Add a count of occurrences for each marketplace
marketplace_counts = raw_data['Marketplace'].value_counts()
marketplace_df['Count'] = marketplace_df['Marketplace'].map(marketplace_counts)

# Sort by count in descending order
marketplace_df = marketplace_df.sort_values('Count', ascending=False)

# Reset index to make it look cleaner
marketplace_df = marketplace_df.reset_index(drop=True)

# Display the table
print("Unique Marketplaces Table:")
print(marketplace_df)

# Optional: If you want a more formatted display, you can use:
# from IPython.display import display
# display(marketplace_df)

Unique Marketplaces Table:
     Marketplace  Count
0   U.S. Private   5991
1    U.S. Public   4111
2   Registration    172
3      Withdrawn    168
4  India Private      1


In [108]:
# Assuming clean_data is your DataFrame
# Group by Issue_Date and Ticker_Symbol, then count occurrences
ticker_by_issue = raw_data.groupby(['Issue_Date', 'Ticker_Symbol']).size().reset_index(name='Count')

# Filter for counts greater than 1
duplicate_tickers = ticker_by_issue[ticker_by_issue['Count'] > 1]

# Sort by Count in descending order for better readability
duplicate_tickers = duplicate_tickers.sort_values('Count', ascending=False)

# Reset index for clean display
duplicate_tickers = duplicate_tickers.reset_index(drop=True)

# Display the table
print("Tickers by Issue Date with Count > 1:")
print(duplicate_tickers)

# Optional: If you want to format the output differently
# print(duplicate_tickers.to_string(index=False))  # Without index
# or
# from IPython.display import display
# display(duplicate_tickers)

Tickers by Issue Date with Count > 1:
     Issue_Date Ticker_Symbol  Count
0    18/11/2009         LBTYA      6
1    04/03/2009             D      5
2    24/07/2003           STZ      4
3    28/02/2002            GM      4
4    25/04/2007          IRLD      4
..          ...           ...    ...
394  11/08/2009          GNMT      2
395  11/07/2007           NEM      2
396  11/06/2008           BRS      2
397  11/06/2007           TAP      2
398  31/12/2008          VNGM      2

[399 rows x 3 columns]


#### Data cleaning and Pre-Processing

In [109]:
# Checking for all columns with missing values
missing_data = raw_data.isnull().sum()
print("Columns with missing values:\n", missing_data[missing_data > 0])

Columns with missing values:
 CUSIP6                               3345
NCUSIP                               3345
CRSP_TICKER                          3471
__Digit_CUSIP                        4619
Ticker_Symbol                         957
                                    ...  
Shareholder_Take_Up_All_Markets_    10324
Shares_Ofrd_as___of_Shs_Out_Bef_     8239
Total_Shares_Offered__mil_           2878
Simul__taneous_Offer_Cusip           9584
Stock_Price_at_Close_of_Offer__0     6828
Length: 83, dtype: int64


In [110]:
# Removing unwanted columns from the dataset.
clean_data = raw_data.loc[raw_data['Marketplace'] == "U.S. Public"] # Removed all observations other than "US Public" companies.
clean_data = clean_data.dropna(subset=["Ticker_Symbol"]) # Removed NAs from the Ticker column.
clean_data = clean_data.dropna(subset=["IPO_Flag__Y_N_"]) # Removed NAs from the IPO flag column, since we are only interested in secondary sale of shares.
clean_data = clean_data.drop_duplicates(subset=["Ticker_Symbol", "Issue_Date"], keep="first") # Removed duplicates by combining ticker and issue date.
clean_data = clean_data.dropna(axis=1, how='all') # Drop column if ALL values are NAs
clean_data = clean_data.reset_index(drop=True).drop(columns=["Issue_Date1", "Issue_Date2" , "Date_Last_Updated" , "Date_Created", "Year" ]) # Remove multiple date columns
clean_data = clean_data.reset_index(drop=True).drop(columns=["CRSP_date", "CUSIP6" , "NCUSIP" ,"CRSP_TICKER", "PERMCO", "PRC" , "SHROUT", "CUSIP" ,"Main_SIC_Code","Master_Deal_Type","Simul__taneous_Offer_Cusip", "__Digit_CUSIP" , "Investor_Cusip" , "Issuer_Borrower_SEDOL", "Deal_Number"]) # Remove multiple identity columns
clean_data = clean_data.reset_index(drop=True).drop(columns=["Payment_Date", "Cur__rency" , "Marketplace" ,"Stock_Price_at_Close_of_Offer__1", "Domicile_Nation_Code"]) 
clean_data['Bookrunner_s_'] = clean_data['Bookrunner_s_'].replace('NOT-AVAILABLE', "Unknown")
clean_data['Bookrunner_s_'] = clean_data['Bookrunner_s_'].replace('NOTAPP', "Unknown")
clean_data['Gross_Spread_per_share_or_bond__'] = clean_data['Gross_Spread_per_share_or_bond__'].replace('na', 0)
clean_data['Gross_Spread_as___of__Prncpl_Amt'] = clean_data['Gross_Spread_as___of__Prncpl_Amt'].replace('na', 0)
clean_data['Filing_Date'] = clean_data['Filing_Date'].replace('.', pd.NA)
clean_data['PERMNO'] = clean_data['PERMNO'].replace('.', pd.NA)
clean_data = clean_data.drop(columns="Over_Subscription_Flag")
clean_data = clean_data.drop(columns="Prices_Update")

# Bulk Drop of all unwanted columns.
columns_to_drop = [
    'Obs', 'Issue_Date3', 'Manage__ment_Fee____',
    'Under__writing_Fee____', 'Selling_Conces__sion____', 'Re__allow__ance_Fee____',
    'Management_Fee_as____of_Prncpl_A', 'Underwriting_Fee_as___of__Prncpl',
    "Settlement_Date",'Selling__Concession__as___of__Pr', 'Reallowance_Fee_as___of_Prncpl_A',
    'Gross__Spread____in_this__Mkt___', 'Principal__Amt___sum__of_all_Mkt',
    'Proceeds__Amt___sum__of_all_Mkts', 'Amt_Filed____in_this_Mkt_____mil',
    'Total_Dollar_Amount_Filed', 'Amended_Secondary_Amount_Filed_T',
    'Amended__Secondary__Shs_Filed___', 'Amended__Sec_Shs__Filed_as____of',
    'Sec_Shrs_Filed_as___of_Shares_Am', 'Amended_Secondary_Shares_Filed',
    'Amended__Shs_Filed____in_this_Mk', 'Shares__Offered____in_this_Mkt1',
    'Original_Middle_of_Filing_Price0', 'Gross__Spread____in_this__Mkt__0',
    'Total_Manage__ment_Fee____', 'Mgmt_as___Gross_Spread', 'Total____Re__allow__ance_Fee____',
    'Total_Re__allow__ance_Fee____mil', 'Total_Under__writing_Fee____mil_',
    'Proceeds_Amt___in__this_Mkt____0', 'Secondary_Amount_Filed_This_Mkt_',
    'Secondary_Amount_Offered_This_Mk', 'Sec_Shs_as____of_Shs__Out_Aft__O',
    'Sec_Shs_as____of_Shs__Out_Bef__O', 'Secondary__Shs_Filed____sum_of__',
    'Sec_Shs__Filed_as____of_Shs__Fil', 'Original_Secondary_Shares_Filed',
    'Secondary__Shs_Ofrd___sum_of__al', 'Secondary__Shs_Ofrd___sum_of_all',
    'Sec_Shs__as___of_Shs__Ofrd___sum', 'Sec_Shs__as___of_Shs_Ofrd___in__',
    'Secondary__Shs_Ofrd____in_this_0', 'Total_Shares_Offered__mil_' , "Date_Filing_Amended"
]
# Drop the columns (ignore errors if some columns are missing)
clean_data = clean_data.drop(columns=columns_to_drop)

In [111]:
clean_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3650 entries, 0 to 3649
Data columns (total 33 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   PERMNO                            3548 non-null   object 
 1   Ticker_Symbol                     3650 non-null   object 
 2   Issue_Date                        3650 non-null   object 
 3   Primary_Exchange_Where_Issuer_s_  3650 non-null   object 
 4   Bookrunner_s_                     3650 non-null   object 
 5   Gross_Spread_per_share_or_bond__  3650 non-null   object 
 6   Gross_Spread_as___of__Prncpl_Amt  3650 non-null   object 
 7   Principal_Amount____mil_          3650 non-null   object 
 8   Proceeds_Amt___in__this_Mkt____m  3650 non-null   object 
 9   Offer_Price                       3650 non-null   object 
 10  Type_of_Security                  3650 non-null   object 
 11  Primary_Exchange_Where_Issue_Wil  3650 non-null   object 
 12  Filing

In [100]:
# remove "/" and "-" from Bookrunner_s_ column
clean_data['Bookrunner_s_'] = clean_data['Bookrunner_s_'].str.replace('/', '')
clean_data['Bookrunner_s_'] = clean_data['Bookrunner_s_'].str.replace('-', '')
clean_data['Bookrunner_s_'] = clean_data['Bookrunner_s_'].str.replace(' ', '')

In [101]:
# Getting all unique Tickers and PERMNOs IDs into a file.
all_tickers = clean_data[["Ticker_Symbol","PERMNO"]].drop_duplicates()
all_tickers.to_csv("all_tickers.csv")

In [102]:
# Pulling all tickers with missing Stock closing Price.
missing_stock_price = clean_data[clean_data['Stock_Price_at_Close_of_Offer__0'].isna()][["Ticker_Symbol"]].drop_duplicates()

# Save the result to a CSV file
missing_stock_price.to_csv("missing_stock_price_1.csv")

In [103]:
# Unique key to aid in merge with other datasets.
clean_data['Name_Key_tic'] = (clean_data['Ticker_Symbol']) + "_" + (clean_data['Issue_Date'].astype(str))
clean_data['Name_Key_PERMNO'] = (clean_data['PERMNO']).astype(str) + "_" + (clean_data['Issue_Date'].astype(str))

In [ ]:
clean_data

### Using Stock close price from WRDS dataset to merge with original dataset - Using Ticker & Issue Date as Key

In [ ]:
# Loading the stock price data.
close_price_1 = pd.read_csv('close_price_data.csv')
close_price_1

In [ ]:
# Remove unwanted columns from the dataset.
close_price_1 = close_price_1.drop(close_price_1.columns[: 9], axis = 1)
close_price_1 = close_price_1.drop(columns="ccmbegdt")
close_price_1

In [89]:
# Rename columns for easy understanding
close_price_1.rename(columns={'prccd': 'stock_close_price_wrds', 'datadate': 'date' , 'tic': 'ticker'}, inplace=True)

# Convert from YYYY-MM-DD to DD/MM/YYYY so that it is consistent with the 'Issue_Date' column in the main dataset.
close_price_1['date'] = pd.to_datetime(close_price_1['date']).dt.strftime('%d/%m/%Y')

In [90]:
# Creating a unique key for the stock price data to merge with the main dataset.
close_price_1['Name_Key_tic'] = (close_price_1['ticker']) + "_" + (close_price_1['date'].astype(str))

In [ ]:
close_price_1

In [92]:
# Merging closing price with the original dataset to fill in NAs
clean_data = pd.merge(clean_data, close_price_1, on='Name_Key_tic' , how = "left")

# Fill in the NAs by using the merged data into the original column.
clean_data['Stock_Price_at_Close_of_Offer__0'] = clean_data['Stock_Price_at_Close_of_Offer__0'].fillna(clean_data['stock_close_price_wrds'])

In [16]:
clean_data = clean_data.drop(columns=["stock_close_price_wrds", "ticker", "date"])

In [ ]:
clean_data

In [17]:
# Extracting data to excel to check if the merge was successful

#clean_data.to_csv("clean_data_post_merge1.csv") 

### Using Stock close price from WRDS dataset to merge with original dataset - Using PERMNO & Issue Date as Key

In [18]:
# Pulling all PERMNO IDs with corresponding missing Stock close price into a file.
missing_stock_price = clean_data[clean_data['Stock_Price_at_Close_of_Offer__0'].isna()][["PERMNO"]].drop_duplicates()

# Save the result to a CSV file
missing_stock_price.to_csv("missing_stock_price_2.csv")

In [ ]:
# Loading the dataset
close_price_2 = pd.read_csv('close_price_data_2.csv')
close_price_2

In [ ]:
# Dropping unwanted columns from the dataset.
close_price_2 = close_price_2.drop(close_price_2.columns[: 4], axis = 1)
close_price_2 = close_price_2.drop(columns= ["ccmbegdt", "LPERMCO" , "LINKDT" , "LINKENDDT" , "iid" , "tic"])
close_price_2['datadate'] = pd.to_datetime(close_price_2['datadate']).dt.strftime('%d/%m/%Y')
close_price_2

In [ ]:
# Creating a unique key to merge with the original dataset. 
close_price_2['Name_Key_PERMNO'] = (close_price_2['LPERMNO']).astype(str) + "_" + (close_price_2['datadate'].astype(str))
close_price_2

In [ ]:
# Merging with the original dataset
clean_data = pd.merge(clean_data, close_price_2, on='Name_Key_PERMNO' , how = "left")
clean_data

In [23]:
# Filling NAs into the original column. 
clean_data['Stock_Price_at_Close_of_Offer__0'] = clean_data['Stock_Price_at_Close_of_Offer__0'].fillna(clean_data['prccd'])

In [24]:
# Extracting data to excel to check if the merge was successful

#Clean_data.to_csv("clean_data_post_merge2.csv") 

In [ ]:
clean_data = clean_data.dropna(subset=["Stock_Price_at_Close_of_Offer__0"])
clean_data

#### Drop all columns with missing data more than 10%

In [ ]:
# Calculate the number of columns it will drop
missing_threshold = 0.1

missing_percent = clean_data.isnull().mean()

columns_to_drop = missing_percent[missing_percent > missing_threshold].index.tolist()

nullvalues = len(set(columns_to_drop))
print(nullvalues)

In [27]:
# Droping columns
clean_data = clean_data.drop(columns=columns_to_drop)

In [ ]:
# Checking for columns with missing values
missing_data = clean_data.isnull().sum()
print("Columns with missing values:\n", missing_data[missing_data > 0])

In [ ]:
clean_data = clean_data.sort_values(by=['Ticker_Symbol', 'Issue_Date'])

# Fill missing Yesterday_s_Stock_Price with the last populated value for each Ticker_Symbol
clean_data['Yesterday_s_Stock_Price'] = clean_data.groupby('Ticker_Symbol')['Yesterday_s_Stock_Price'].ffill()


clean_data = clean_data.sort_values(by=['Ticker_Symbol', 'Filing_Date'])
# Fill missing Yesterday_s_Stock_Price with the last populated value for each Ticker_Symbol
clean_data['Filing_Date'] = clean_data.groupby('Ticker_Symbol')['Filing_Date'].ffill()

clean_data = clean_data.sort_values(by=['Ticker_Symbol', 'PERMNO'])

# Fill missing Yesterday_s_Stock_Price with the last populated value for each Ticker_Symbol
clean_data['PERMNO'] = clean_data.groupby('Ticker_Symbol')['PERMNO'].ffill()

missing_data = clean_data.isnull().sum()
print("Columns with missing values:\n", missing_data[missing_data > 0])

In [30]:
#Convert columns to numeric
clean_data['Yesterday_s_Stock_Price'] = pd.to_numeric(clean_data['Yesterday_s_Stock_Price'], errors='coerce')
clean_data['Principal_Amount____mil_'] = pd.to_numeric(clean_data['Principal_Amount____mil_'], errors='coerce')
clean_data['Common_Equity_Before_the_Offerin'] = pd.to_numeric(clean_data['Common_Equity_Before_the_Offerin'], errors='coerce')
clean_data['Deal_Size__as_Pct_of_Market_Cap'] = pd.to_numeric(clean_data['Deal_Size__as_Pct_of_Market_Cap'], errors='coerce')

# Estimate Shares Outstanding (absolute value for negatives)
clean_data['Shares_Outstanding'] = (clean_data['Common_Equity_Before_the_Offerin'] / clean_data['Yesterday_s_Stock_Price']).abs()

# Calculate Market Cap (in millions USD)
clean_data['Market_Cap'] = clean_data['Shares_Outstanding'] * clean_data['Yesterday_s_Stock_Price']

# Calculate Deal_Size__as_Pct_of_Market_Cap
clean_data['Calculated_Deal_Size_as_Pct_of_Market_Cap'] = (clean_data['Principal_Amount____mil_'] / clean_data['Market_Cap']) * 100

# Fill missing values in Deal_Size__as_Pct_of_Market_Cap with the calculated values
clean_data['Deal_Size__as_Pct_of_Market_Cap'] = clean_data['Deal_Size__as_Pct_of_Market_Cap'].fillna(clean_data['Calculated_Deal_Size_as_Pct_of_Market_Cap'])

In [31]:
# Dropping unwanted columns
clean_data = clean_data.drop(columns=["Original_Middle_of_Filing_Price_"])
clean_data = clean_data.dropna(subset=["Shares_Outstanding"])

In [ ]:
# Checking for remaining columns with misssing values
missing_data = clean_data.isnull().sum()
print("Columns with missing values:\n", missing_data[missing_data > 0])

In [33]:
# Dropping rows with missing values as it does not make siginificant portion of the dataset
clean_data = clean_data.dropna()

In [ ]:
# Checking for remaining columns with misssing values
missing_data = clean_data.isnull().sum()
print("Columns with missing values:\n", missing_data[missing_data > 0])

In [35]:
# Replace missing value indicators
missing_indicators = ['na', '.', '']
clean_data.replace(missing_indicators, pd.NA, inplace=True)

# Force converting columns to numeric
numeric_cols = ['Gross_Spread_per_share_or_bond__', 'Gross_Spread_as___of__Prncpl_Amt', 
                'Principal_Amount____mil_', 'Proceeds_Amt___in__this_Mkt____m', 'Offer_Price', 
                'Shares__Offered____in_this_Mkt', 'Yesterday_s_Stock_Price', 
                'Common_Equity_Before_the_Offerin', 'Overallot__Amt_Option___in_this_', 
                'Price_Current', 'Stock_Price_at_Close_of_Offer__0', 'Shares_Outstanding', 
                'Market_Cap', 'Calculated_Deal_Size_as_Pct_of_Market_Cap']
for col in numeric_cols:
    clean_data[col] = pd.to_numeric(clean_data[col], errors='coerce')

#### Adding Additional Datasets - 1-Month US Interest Rate

In [ ]:
interest_rates = pd.read_csv('interest_rates.csv')
interest_rates

In [ ]:
interest_rates['observation_date'] = pd.to_datetime(interest_rates['observation_date']).dt.strftime('%d/%m/%Y')
interest_rates

In [ ]:
clean_data = pd.merge(clean_data, interest_rates, left_on='Issue_Date' ,right_on= "observation_date", how = "left")
clean_data

In [39]:
clean_data.rename(columns={'DGS1MO': '1_month_interest_rate'}, inplace=True)
clean_data = clean_data.drop(columns="observation_date")

In [ ]:
# Fill DGS1MO using forward fill after sorting my ticker and issue date.
clean_data = clean_data.sort_values(by=['Issue_Date'])  # Sort by date
clean_data['1_month_interest_rate'] = clean_data['1_month_interest_rate'].ffill()  # Forward-fill only DGS1MO
clean_data

In [ ]:
#check for any missing values
missing_data = clean_data.isnull().sum()
print("Columns with missing values:\n", missing_data[missing_data > 0])

In [42]:
# Export dataset to csv
clean_data.to_csv('clean_data_post_merge3.csv')

#### Adding Additional Datasets - S&P 500 Data

In [43]:
# Load excel format S&P 500 data
sp500 = pd.read_excel('S&P500_capital_IQ.xlsx')


In [ ]:
# Converting time into a format similar to the original dataset so that merging is easy
sp500['Pricing Date'] = pd.to_datetime(sp500['Pricing Date']).dt.strftime('%d/%m/%Y')
sp500

In [45]:
# Merge the data using the date column
clean_data = pd.merge(clean_data, sp500, left_on='Issue_Date' ,right_on= "Pricing Date", how = "left")

In [ ]:
#check for any missing values
missing_data = clean_data.isnull().sum()
print("Columns with missing values:\n", missing_data[missing_data > 0])

In [ ]:
# Sort the data by date and forward fill for missing values
clean_data = clean_data.sort_values(by=['Issue_Date'])
clean_data['S&P 500 Price Return-Index Value (Daily)'] = clean_data['S&P 500 Price Return-Index Value (Daily)'].ffill()
clean_data

#### Adding Additional Datasets - Unemployment Rate Data

In [48]:
# Loading unemployement data
unemployment = pd.read_csv('unemployment_rate_FRED.csv')

In [ ]:
# Converting time into a format similar to the original dataset so that merging is easy
unemployment['observation_date'] = pd.to_datetime(unemployment['observation_date']).dt.strftime('%d/%m/%Y')
unemployment

In [ ]:
# Creating a new column "Year" from the observation_date column
unemployment['Year'] = pd.to_datetime(unemployment['observation_date']).dt.year
unemployment

In [ ]:
# Spliting issue date by its delimiter into 3 columns
clean_data[['Day', 'Month', 'Year']] = clean_data['Issue_Date'].str.split('/', expand=True)
clean_data

# Droping day and month columns
clean_data = clean_data.drop(columns=['Day', 'Month'])
clean_data

In [52]:
# convert Year column in both datasets as object
unemployment['Year'] = unemployment['Year'].astype(str)
clean_data['Year'] = clean_data['Year'].astype(str)

In [ ]:
# merge the unemployment data with the full dataset
clean_data = pd.merge(clean_data, unemployment, left_on='Year' ,right_on= "Year", how = "left")
clean_data

In [54]:
clean_data.rename(columns={'UNRATE': '1_month_interest_rate'}, inplace=True)

In [ ]:
# check for missing values
missing_data = clean_data.isnull().sum()
print("Columns with missing values:\n", missing_data[missing_data > 0])

#### Adding Additional Datasets - GDP per Capita

In [ ]:
# Loading GDP dataset
gdp = pd.read_csv('gdp_per_capita.csv')
gdp.info()

In [ ]:
# convert obeeravtion_date to only year format
gdp['observation_date'] = pd.to_datetime(gdp['observation_date']).dt.strftime('%Y')
gdp

In [ ]:
# merge the gdp data with the full dataset
clean_data = pd.merge(clean_data, gdp, left_on='Year' ,right_on= "observation_date", how = "left")
clean_data

In [59]:
clean_data.rename(columns={'A939RC0Q052SBEA': 'GDP_per_capita'}, inplace=True)

In [ ]:
# check for missing values
missing_data = clean_data.isnull().sum()
print("Columns with missing values:\n", missing_data[missing_data > 0])

#### Finalizing the Dataset

In [61]:
# Create a list of columns to drop
columns_to_drop = ['Name_Key_tic', 'Name_Key_PERMNO', "Common_Equity_Before_the_Offerin", 
                   'observation_date_x', 'Pricing Date', "observation_date_y",
                   "PERMNO" , "Ticker_Symbol", "Filing_Date", "Calculated_Deal_Size_as_Pct_of_Market_Cap"]

clean_data = clean_data.drop(columns=columns_to_drop)

In [62]:
# Renaming columns for better understanding
#clean_data.rename(columns={'DGS1MO': '1_month_interest_rate'}, inplace=True)

Calculating Underpricing

In [63]:
clean_data['Underpricing'] = (clean_data['Stock_Price_at_Close_of_Offer__0'] - clean_data['Offer_Price']) / clean_data['Offer_Price']

Creating Underpricing Flag

In [ ]:
# create uderpricing flag column. If underpricing is greater than 0.15%, then the flag is 1, else 0
clean_data['Underpricing_Flag'] = (clean_data['Underpricing'] > 0.03).astype(int)
clean_data = clean_data.drop(columns="Underpricing")

# reset index
clean_data = clean_data.reset_index(drop=True)
clean_data

In [65]:
clean_data.to_csv('final_clean_dataset_assignment_2.csv')

In [112]:
clean_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3650 entries, 0 to 3649
Data columns (total 33 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   PERMNO                            3548 non-null   object 
 1   Ticker_Symbol                     3650 non-null   object 
 2   Issue_Date                        3650 non-null   object 
 3   Primary_Exchange_Where_Issuer_s_  3650 non-null   object 
 4   Bookrunner_s_                     3650 non-null   object 
 5   Gross_Spread_per_share_or_bond__  3650 non-null   object 
 6   Gross_Spread_as___of__Prncpl_Amt  3650 non-null   object 
 7   Principal_Amount____mil_          3650 non-null   object 
 8   Proceeds_Amt___in__this_Mkt____m  3650 non-null   object 
 9   Offer_Price                       3650 non-null   object 
 10  Type_of_Security                  3650 non-null   object 
 11  Primary_Exchange_Where_Issue_Wil  3650 non-null   object 
 12  Filing

#### PCA for Feature Selection

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Step 1: Select only numeric columns
numeric_cols = clean_data.select_dtypes(include=['number'])

# Step 2: Log-transform the numeric columns
clean_data_log_transformed = numeric_cols.apply(np.log1p)

# Step 3: Check if any NaN values were introduced after log transformation
if clean_data_log_transformed.isna().sum().any():
    print("NaN values found in the log-transformed data:")
    print(clean_data_log_transformed.isna().sum())
    
    # You can choose to drop rows or fill NaNs
    clean_data_log_transformed = clean_data_log_transformed.dropna()  # or .fillna(0), depending on your choice

# Step 4: Standardize the log-transformed data
scaler = StandardScaler()
numeric_scaled = scaler.fit_transform(clean_data_log_transformed)

# Step 5: Apply PCA
pca = PCA()
pca_components = pca.fit_transform(numeric_scaled)

# Step 6: Explained variance
explained_variance = pca.explained_variance_ratio_

# Step 7: Plot explained variance
plt.figure(figsize=(8, 6))
plt.plot(range(1, len(explained_variance) + 1), explained_variance, marker='o')
plt.title('Explained Variance by Each Principal Component')
plt.xlabel('Principal Components')
plt.ylabel('Explained Variance')
plt.show()

# Step 8: Create a DataFrame for the principal components (optional)
pca_df = pd.DataFrame(pca_components, columns=[f'PC{i+1}' for i in range(pca_components.shape[1])])

# Step 9: Display results
print("Explained Variance by Component: ", explained_variance)
print("Principal Components Data: ")
print(pca_df.head())


In [ ]:
# Step 10: Extract the top 3 principal components' loadings (variables' contributions)
# The components_ attribute contains the loadings of the original features for each principal component
loadings = pca.components_

# Display the variables contributing to the first 3 principal components
for i in range(3):
    print(f"\nTop contributing variables to PC{i+1}:")
    # Create a DataFrame for easy viewing of loadings with feature names
    loading_df = pd.DataFrame(loadings[i], index=numeric_cols.columns, columns=[f"PC{i+1}"])
    # Sort the loadings by the absolute value, showing the highest contributing features
    sorted_loading_df = loading_df.abs().sort_values(by=f"PC{i+1}", ascending=False)
    print(sorted_loading_df.head(10))  # Display top 10 contributing features for each PC

In [ ]:
import seaborn as sns

# Step 1: Extract absolute values of loadings
loadings_df = pd.DataFrame(pca.components_, columns=numeric_cols.columns)

# Step 2: Select top 3 principal components
top_pcs = loadings_df.iloc[:3].T  # Transpose to get features as rows

# Step 3: Compute absolute values for feature importance
top_pcs_abs = top_pcs.abs()

# Step 4: Plot the feature importance
plt.figure(figsize=(10, 6))
sns.heatmap(top_pcs_abs, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
plt.title("Feature Importance in Top 3 Principal Components")
plt.xlabel("Principal Components")
plt.ylabel("Features")
plt.show()


In [ ]:
# Step 1: Extract PCA loadings
loadings_df = pd.DataFrame(pca.components_, columns=numeric_cols.columns)

# Step 2: Select top 3 principal components and transpose
top_pcs = loadings_df.iloc[:3].T  

# Step 3: Plot with sign to see direction
plt.figure(figsize=(12, 6))
sns.heatmap(top_pcs, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5, center=0)
plt.title("Feature Influence (Direction) in Top 3 Principal Components")
plt.xlabel("Principal Components")
plt.ylabel("Features")
plt.show()


#### Taking PCAs and Running Supervised Machine Learning Models

In [ ]:
# Step 2: Apply PCA and take the first 3 principal components
pca_3_components = pca.transform(numeric_scaled)[:, :3]

# Step 3: Create a DataFrame with the first 3 principal components
pca_df = pd.DataFrame(pca_3_components, columns=['PC1', 'PC2', 'PC3'])

# Step 4: Prepare target variable 'y'
y = clean_data['Underpricing_Flag']  # Assuming this is the target variable

# Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(pca_df, y, test_size=0.2, random_state=42)

# Models to evaluate
models = {
    'Random Forest': RandomForestClassifier(random_state=42),
    'KNN': KNeighborsClassifier(),
    'Logistic Regression': LogisticRegression(random_state=42),
    'LightGBM': LGBMClassifier(random_state=42),
    'XGBoost': XGBClassifier(random_state=42)
}

# Initialize results list
results = []

# Step 6: Train and evaluate the models
for model_name, model in models.items():
    print(f"\nEvaluating {model_name}...")
    
    # Fit the model on the training data
    model.fit(X_train, y_train)
    
    # Make predictions on the test data
    y_pred = model.predict(X_test)
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
    
    # Print the metrics
    print(f"{model_name} - Accuracy: {accuracy:.4f}")
    print(f"{model_name} - Recall: {recall:.4f}")
    print(f"{model_name} - Precision: {precision:.4f}")
    print(f"{model_name} - ROC AUC: {roc_auc:.4f}")
    
    # Append results
    results.append([model_name, accuracy, recall, precision, roc_auc])

In [ ]:
# Step 7: Create comparison table
results_df = pd.DataFrame(
    results,
    columns=['Model', 'Accuracy', 'Recall', 'Precision', 'ROC AUC']
)
print("\nModel Performance Comparison Table:")
print(results_df.round(4))

In [ ]:
# Step 7: Create comparison table
results_df = pd.DataFrame(
    results,
    columns=['Model', 'Accuracy', 'Recall', 'Precision', 'ROC AUC']
)
print("\nModel Performance Comparison Table:")
print(results_df.round(4))

#### Adjusting Threshold Value - Finding optimal

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# Function to calculate metrics at different thresholds
def get_metrics_at_thresholds(model, X_test, y_test):
    # Get predicted probabilities
    y_pred_prob = model.predict_proba(X_test)[:, 1]
    
    # Calculate ROC curve
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
    
    # Calculate Youden's J statistic (tpr - fpr)
    j_scores = tpr - fpr
    
    # Find the optimal threshold based on Youden's J statistic
    optimal_threshold = thresholds[np.argmax(j_scores)]
    
    # Calculate the performance at the optimal threshold
    y_pred_optimal = (y_pred_prob >= optimal_threshold).astype(int)
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred_optimal)
    recall = recall_score(y_test, y_pred_optimal)
    precision = precision_score(y_test, y_pred_optimal)
    roc_auc = roc_auc_score(y_test, y_pred_prob)
    
    return accuracy, recall, precision, roc_auc, optimal_threshold, j_scores, fpr, tpr, thresholds

# Ensure X_train_resampled and y_train_resampled exist, or use X_train, y_train
try:
    X_train_resampled, y_train_resampled
except NameError:
    X_train_resampled, y_train_resampled = X_train, y_train  # Use original split if resampled is missing

# Loop through each model and calculate metrics at the optimal threshold
for model_name, model in models.items():
    print(f"\nEvaluating {model_name} at optimal ROC threshold...")

    # Fit the model on the training data
    model.fit(X_train_resampled, y_train_resampled)
    
    # Get metrics and the optimal threshold
    accuracy, recall, precision, roc_auc, optimal_threshold, j_scores, fpr, tpr, thresholds = get_metrics_at_thresholds(model, X_test, y_test)
    
    # Print the metrics at the optimal threshold
    print(f"{model_name} - Optimal Threshold: {optimal_threshold:.4f}")

#### Running Models

In [ ]:
# Step 1: Apply PCA and take the first 3 principal components
pca_3_components = pca.transform(numeric_scaled)[:, :3]

# Step 2: Create a DataFrame with the first 3 principal components
pca_df = pd.DataFrame(pca_3_components, columns=['PC1', 'PC2', 'PC3'])

# Step 3: Prepare target variable 'y'
y = clean_data['Underpricing_Flag']  # Replace with your actual target if different

# Step 4: Train-test split (80/20)
X_train_full, X_test, y_train_full, y_test = train_test_split(
    pca_df, y, test_size=0.2, random_state=42, stratify=y
)

# Define models
models = {
    'Random Forest': RandomForestClassifier(random_state=42),
    'KNN': KNeighborsClassifier(),
    'Logistic Regression': LogisticRegression(random_state=42),
    'LightGBM': LGBMClassifier(random_state=42),
    'XGBoost': XGBClassifier(random_state=42)
}

# Function to get optimal threshold using Youden's J statistic
def get_metrics_at_thresholds(model, X, y):
    y_pred_prob = model.predict_proba(X)[:, 1]
    fpr, tpr, thresholds = roc_curve(y, y_pred_prob)
    j_scores = tpr - fpr
    optimal_idx = np.argmax(j_scores)
    optimal_threshold = thresholds[optimal_idx]
    return optimal_threshold, y_pred_prob, fpr, tpr  # Return all for ROC plotting

# Step 5: 5-fold CV with SMOTE to determine optimal thresholds
skf = StratifiedKFold(n_splits=20)
optimal_thresholds = {}

for model_name, model in models.items():
    print(f"\nEvaluating {model_name} with 5-fold CV and SMOTE...")
    thresholds_per_fold = []
    
    for train_idx, val_idx in skf.split(X_train_full, y_train_full):
        X_train = X_train_full.iloc[train_idx]
        X_val = X_train_full.iloc[val_idx]
        y_train = y_train_full.iloc[train_idx]
        y_val = y_train_full.iloc[val_idx]
        
        # Apply SMOTE to training data within the fold
        smote = SMOTE(random_state=42)
        X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
        
        # Fit the model
        model.fit(X_train_resampled, y_train_resampled)
        
        # Get optimal threshold
        optimal_threshold, _, _, _ = get_metrics_at_thresholds(model, X_val, y_val)
        thresholds_per_fold.append(optimal_threshold)
    
    # Store the average optimal threshold
    optimal_thresholds[model_name] = np.mean(thresholds_per_fold)
    print(f"{model_name} - Average Optimal Threshold from CV: {optimal_thresholds[model_name]:.4f}")

# Step 6: Evaluate on test data using stored optimal thresholds
results = []
plt.figure(figsize=(10, 6))

for model_name, model in models.items():
    print(f"\nEvaluating {model_name} on Test Data with Optimal Threshold...")
    
    # Retrain on full SMOTE-balanced training set
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train_full, y_train_full)
    model.fit(X_train_resampled, y_train_resampled)
    
    # Use the optimal threshold
    optimal_threshold = optimal_thresholds[model_name]
    
    # Get predictions
    y_pred_prob = model.predict_proba(X_test)[:, 1]
    y_pred_optimal = (y_pred_prob >= optimal_threshold).astype(int)
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred_optimal)
    recall = recall_score(y_test, y_pred_optimal)
    precision = precision_score(y_test, y_pred_optimal)
    roc_auc = roc_auc_score(y_test, y_pred_prob)
    
    # Store results
    results.append([model_name, accuracy, recall, precision, roc_auc])
    
    # Print metrics
    print(f"{model_name} - Optimal Threshold: {optimal_threshold:.4f}")
    print(f"{model_name} - Accuracy: {accuracy:.4f}")
    print(f"{model_name} - Recall: {recall:.4f}")
    print(f"{model_name} - Precision: {precision:.4f}")
    print(f"{model_name} - ROC AUC: {roc_auc:.4f}")
    
    # Print confusion matrix
    cm = confusion_matrix(y_test, y_pred_optimal)
    print(f"Confusion Matrix for {model_name}:\n{cm}")
    
    # Plot ROC curve
    _, _, fpr, tpr = get_metrics_at_thresholds(model, X_test, y_test)
    plt.plot(fpr, tpr, label=f'{model_name} (AUC = {roc_auc:.2f})')

# Finalize ROC plot
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve Comparison')
plt.legend()
plt.show()



In [ ]:
# Step 7: Create comparison table
results_df = pd.DataFrame(
    results,
    columns=['Model', 'Accuracy', 'Recall', 'Precision', 'ROC AUC']
)
print("\nModel Performance Comparison Table:")
print(results_df.round(4))

In [ ]:
#print confusion matrix for each model
from sklearn.metrics import confusion_matrix

for model_name, model in models.items():
    print(f"\nConfusion matrix for {model_name}:")
    print(confusion_matrix(y_test, model.predict(X_test)))